In [17]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [18]:
chain.predict(question="My name is Shin Woo Lee.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Shin Woo Lee.

> Finished chain.


'Nice to meet you, Shin Woo Lee! How can I assist you today?'

In [19]:
chain.predict(question="I live in Seoul.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Shin Woo Lee.
AI: Nice to meet you, Shin Woo Lee! How can I assist you today?
Human: I live in Seoul.

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'

In [20]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Shin Woo Lee.
AI: Nice to meet you, Shin Woo Lee! How can I assist you today?
Human: I live in Seoul.
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is Shin Woo Lee. How can I assist you further, Shin Woo Lee?'

In [21]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Shin Woo Lee.'),
  AIMessage(content='Nice to meet you, Shin Woo Lee! How can I assist you today?'),
  HumanMessage(content='I live in Seoul.'),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Shin Woo Lee. How can I assist you further, Shin Woo Lee?')]}